# Memory experiment on an extended logical qubit

This notebook aims at performing a memory experiment on an extended logical qubit (see image below).

![A logical qubit that has been extended horizontally by 2 plaquettes](./images/extended_logical_qubit.png)


## Mandatory imports


In [ ]:
import cirq
import matplotlib.pyplot as plt
import sinter
import stim
from stimcirq import cirq_circuit_to_stim_circuit

from tqec.templates.constructions.qubit import QubitRectangleTemplate
from tqec.templates.scale import LinearFunction
from tqec.circuit.operations.operation import make_shift_coords, make_observable
from tqec.circuit.operations.transformer import transform_to_stimcirq_compatible
from tqec.enums import PlaquetteOrientation
from tqec.circuit.circuit import generate_circuit
from tqec.noise_models import (
    AfterCliffordDepolarizingNoise,
    AfterResetFlipNoise,
    BeforeMeasurementFlipNoise,
)
from tqec.plaquette.plaquette import Plaquette
from tqec.plaquette.library import (
    z_initialisation_square_plaquette,
    z_initialisation_rounded_plaquette,
    xx_memory_plaquette,
    xxxx_memory_plaquette,
    zz_memory_plaquette,
    zzzz_memory_plaquette,
    measurement_rounded_plaquette,
    measurement_square_plaquette,
)
from tqec.templates.display import display_template

## Normalisation and noisyness

Once the quantum error correction circuit implemented, we still need to apply two passes to obtain a circuit ready to be translated by the `stimcirq`.

The first pass normalises the `cirq.Circuit` produced. This pass was performing several modifications before, but is now simply removing potential empty `cirq.Moment` instances from the `cirq.Circuit` instance.

The second pass applies the noise model(s) we want to consider in the `stim` simulation.


In [ ]:
def normalise_circuit(circuit: cirq.Circuit) -> cirq.Circuit:
    ordered_transformers = [
        cirq.drop_empty_moments,
    ]
    for transformer in ordered_transformers:
        circuit = transformer(circuit)
    return circuit


def to_noisy_circuit(circuit: cirq.Circuit, noise_level: float) -> cirq.Circuit:
    noise_models = [
        AfterCliffordDepolarizingNoise(noise_level),
        AfterResetFlipNoise(noise_level),
        BeforeMeasurementFlipNoise(noise_level),
    ]
    for nm in noise_models:
        circuit = circuit.with_noise(nm)
    return circuit

## Building the Circuit instance

The main goal of the `tqec` library is to construct the `cirq.Circuit` instance representing the QEC experiment we are interested in. In this notebook, we want to perform a memory experiment on an "extended" logical qubit.

The different pre-defined Plaquette instances `XXPlaquetteList`, ... implement by default the memory experiment, so we can use them without worrying about the quantum circuits actually executed.


In [ ]:
width, height = LinearFunction(4), LinearFunction(2)
template = QubitRectangleTemplate(width, height)
display_template(template)

In [ ]:
from tqec.circuit.observable_qubits import observable_qubits_from_template
from tqec.templates.display import display_templates_ascii


def generate_cirq_circuit_tqec(k: int, repetitions: int) -> cirq.Circuit:
    # 1. Define the template.
    # Here, the template is directly constructed to the requested dimensions, so we do not need to
    # re-scale it using the Template.to_scale method.
    template = QubitRectangleTemplate(LinearFunction(4), LinearFunction(2), k=k)
    # 2. Define the different plaquettes that will be used on the template defined above.
    # As noted in the text description above this function, the plaquette instances in the list below
    # implement by default the memory experiment we want to perform. That is the reason why there is no
    # tedious quantum circuit definition here.
    # The following time schedules are used across plaquettes:
    # - Initial reset index is 1
    # - (H gate for X-stabilizers is 2)
    # - CNOTs indices are 3, 4, 5, 6
    # - (H gate for X-stabilizers is 7)
    # - Measurement index is 8
    plaquettes: list[list[Plaquette]] = [
        [
            z_initialisation_rounded_plaquette(PlaquetteOrientation.UP),
            xx_memory_plaquette(
                PlaquetteOrientation.UP,
                [1, 2, 5, 6, 7, 8],
                include_detector=False,
                is_first_round=True,
            ),
            xx_memory_plaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            measurement_rounded_plaquette(
                PlaquetteOrientation.UP, include_detector=False
            ),
        ],
        [
            z_initialisation_rounded_plaquette(PlaquetteOrientation.LEFT),
            zz_memory_plaquette(
                PlaquetteOrientation.LEFT, [1, 5, 6, 8], is_first_round=True
            ),
            zz_memory_plaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            measurement_rounded_plaquette(PlaquetteOrientation.LEFT),
        ],
        [
            z_initialisation_square_plaquette(),
            xxxx_memory_plaquette(
                [1, 2, 3, 4, 5, 6, 7, 8], include_detector=False, is_first_round=True
            ),
            xxxx_memory_plaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            measurement_square_plaquette(include_detector=False),
        ],
        [
            z_initialisation_square_plaquette(),
            zzzz_memory_plaquette([1, 3, 4, 5, 6, 8], is_first_round=True),
            zzzz_memory_plaquette([1, 3, 4, 5, 6, 8]),
            measurement_square_plaquette(),
        ],
        [
            z_initialisation_rounded_plaquette(PlaquetteOrientation.RIGHT),
            zz_memory_plaquette(
                PlaquetteOrientation.RIGHT, [1, 3, 4, 8], is_first_round=True
            ),
            zz_memory_plaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            measurement_rounded_plaquette(PlaquetteOrientation.RIGHT),
        ],
        [
            z_initialisation_rounded_plaquette(PlaquetteOrientation.DOWN),
            xx_memory_plaquette(
                PlaquetteOrientation.DOWN,
                [1, 2, 3, 4, 7, 8],
                include_detector=False,
                is_first_round=True,
            ),
            xx_memory_plaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
            measurement_rounded_plaquette(
                PlaquetteOrientation.DOWN, include_detector=False
            ),
        ],
    ]

    # 3. Define the layer modifiers.
    # In the memory experiment, we want to repeat the second layer `repetitions` times in order to
    # let the experiment run for an extended period of time.
    # This is done by encapsulating the circuit generated for the second layer into a
    # cirq.CircuitOperation that natively supports repeating an operation efficiently, and that is
    # natively recognized by the `tqec` library.
    def make_repeated_layer(circuit: cirq.Circuit) -> cirq.Circuit:
        circuit_to_repeat = circuit + cirq.Circuit(
            cirq.Moment(make_shift_coords(0, 0, 1))
        )
        repeated_circuit_operation = cirq.CircuitOperation(
            circuit_to_repeat.freeze()
        ).repeat(repetitions)
        return cirq.Circuit([repeated_circuit_operation])

    layer_modificators = {2: make_repeated_layer}

    # 4. Actually create the cirq.Circuit instance by concatenating the circuits generated
    # for each layer and potentially modified by the modifiers defined above.
    circuit = cirq.Circuit()
    for layer_index in range(4):
        layer_circuit = generate_circuit(
            template,
            [plaquette_list[layer_index] for plaquette_list in plaquettes],
        )
        layer_circuit = normalise_circuit(layer_circuit)
        circuit += layer_modificators.get(layer_index, lambda circ: circ)(layer_circuit)
        circuit += cirq.Moment(make_shift_coords(0, 0, 1))

    # 5. Define the observable.
    # The observable is defined and added to the cirq.Circuit instance just here.
    increments = template.get_increments()
    origin = cirq.GridQubit(increments.y * k + 1, 1)
    circuit.append(
        cirq.Moment(
            make_observable(
                origin,
                [
                    (cirq.GridQubit(0, i * increments.x), -1)
                    for i in range(0, template.shape.x - 1)
                ],
            )
        )
    )

    # 6. Due to limitations in the API provided by Stim, the DetectorGate and ObservableGate instances
    # in the circuit only contain local measurement records, an internal representation that is not
    # understood by Stim. The fill_in_global_record_indices function replaces these local measurement
    # records by global ones that are understood by Stim.
    circuit_with_detectors = transform_to_stimcirq_compatible(circuit)
    return circuit_with_detectors

## Include noise level and a better API

In order to perform the experiment and to compute the error threshold for such an experiment, we need to be able to vary the noise applied to our circuit. Also, we would like to only have one input that controls both the width and the height, to be able to vary only that input.

The following function builds a template that is twice as large horizontally than vertically and applies the provided noise level to the whole circuit before translating the circuit to `Stim`.


In [ ]:
def generate_stim_circuit_tqec(
    code_distance: int, noise_level: float, repetitions: int
) -> stim.Circuit:
    k = (code_distance - 1) // 2
    circuit = generate_cirq_circuit_tqec(k, repetitions)
    if abs(noise_level) > 1e-12:
        circuit = to_noisy_circuit(circuit, noise_level)
    else:
        print(
            f"Warning: a noise level below 1e-12 ({noise_level}) has been detected. Noise is disabled. If this was intentional, please remove this code."
        )

    return cirq_circuit_to_stim_circuit(circuit)

## TQEC plots

We should finally be ready to perform the `stim` simulations and plot the results of our `tqec`-generated QEC memory experiment.


In [ ]:
surface_code_tasks_tqec = [
    sinter.Task(
        circuit=generate_stim_circuit_tqec(d, noise, 3 * d),
        json_metadata={"d": d, "r": 3 * d, "p": noise},
    )
    for d in [3, 5, 7, 9, 11]
    for noise in [0.001, 0.002, 0.005, 0.01, 0.012, 0.014]
]

collected_surface_code_stats_tqec: list[sinter.TaskStats] = sinter.collect(
    num_workers=20,
    tasks=surface_code_tasks_tqec,
    decoders=["pymatching"],
    max_shots=1_000_000,
    max_errors=5_000,
    print_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats_tqec,
    x_func=lambda stat: stat.json_metadata["p"],
    group_func=lambda stat: stat.json_metadata["d"],
    failure_units_per_shot_func=lambda stat: stat.json_metadata["r"],
)
ax.loglog()
ax.set_title("TQEC: Surface Code Error Rates per Round under Circuit Noise")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Round")
ax.grid(which="major")
ax.grid(which="minor")
ax.legend()
fig.set_dpi(120)  # Show it bigger

## Circuit visualisation

Some debugging relicates that are left here if you want to visualise/compare the quantum circuits generated.


In [ ]:
stim_circuit_tqec = generate_stim_circuit_tqec(3, 0.001, 2)
stim_circuit_tqec.diagram("timeslice-svg")